In [1]:
import numpy as np
import pandas as pd
from sdeint import itoint
import plotly.express as px

# Single Pendulum Control

In [2]:
errors_dict = {0: 0}
control_signal = {0: 0}
def u(y, t, setpoint, Kp, Kd, Ki):
    """Pendulum control law
    Args:
        y (n x 1 np.ndarray): [theta, omega]
        t (float): time
        setpoint (float, optional): setpoint. Defaults to np.pi/4.
        Kp (float, optional): proportional gain. Defaults to 1.
        Kd (float, optional): derivative gain. Defaults to 0.1.
        Ki (float, optional): integral gain. Defaults to 0.1.
    Returns:
        float: control input
    """
    theta, omega = y
    e = setpoint - theta
    errors_list = list(errors_dict.values())
    e_dot = (e - errors_list[-1]) / dt
    errors_dict[t] = e
    control_signal[t] = Kp*e + Kd*e_dot + Ki*np.sum(errors_list)*dt
    return control_signal[t]

def f(y, t, c, u):
    """Pendulum ODE
    Args:
        y (n x 1 np.ndarray): [theta, omega]
        t (float): time
        u (float): control input
    Returns:
        n x 1 np.ndarray: [dtheta/dt, domega/dt]
    """
    return np.array([y[1], -c * y[1] - np.sin(y[0]) + u])
    # return np.array([y[1], -np.sin(y[0])])


def G(y, t, epsilon):
    """Pendulum diffusion matrix
    Args:
        y (n x 1 np.ndarray): [theta, omega]
        t (float): time
        epsilon (float, optional): maximum diffusion strength. Defaults to 0.1.
    Returns:
        n x n np.ndarray: diffusion matrix
    """
    n = y.shape[0]
    return np.random.uniform(-epsilon, epsilon, size=(n, n))


# pendulum initial conditions
y0 = np.array([np.pi/2, 0])

# pendulum time span
dt = 0.1
tspan = np.arange(0, 100, dt).round(2)

# parameters
SETPOINT = np.pi

KP = 7.2  # proportional gain
KD = 8  # derivative gain 
KI = 0.03  # integral gain
MAX_DIFFUSION = 0.1  # maximum diffusion strength
C = 1e-3  # damping

ito = itoint(
    f=lambda y, t: f(y, t, C, u(y, t, SETPOINT, KP, KD, KI)),
    G=lambda y, t: G(y, t, MAX_DIFFUSION),
    y0=y0,
    tspan=tspan)

df = pd.DataFrame(tspan, columns=['t'])
df[['theta', 'omega']] = ito
df['x'] = np.sin(df['theta'])
df['y'] = -np.cos(df['theta'])
df['x_dot'] = df['omega'] * np.cos(df['theta'])
df['y_dot'] = df['omega'] * np.sin(df['theta'])
df['error'] = errors_dict.values()
df['control_signal'] = control_signal.values()
df['setpoint'] = SETPOINT

# pendulum theta, omega plot
px.line(df, x='t', y=['theta', 'setpoint']).update_layout(title='Pendulum', xaxis_title='t', yaxis_title='theta, omega',
                                                          template='plotly_dark', paper_bgcolor='#1f1f1f', plot_bgcolor='#181818').show()

# pendulum phase plot
# px.scatter(df, x='theta', y='omega', color='t').update_layout(title='Pendulum Phase', xaxis_title='theta', yaxis_title='omega',
#                                         template='plotly_dark', paper_bgcolor='#1f1f1f', plot_bgcolor='#181818').update_traces(marker=dict(size=2)).show()

# animate x and y with plotly
df_reduced = df.iloc[::2, :]
fig = px.scatter(df_reduced, x='x', y='y', animation_frame='t').update_layout(
    title='Pendulum', template='plotly_dark', paper_bgcolor='#1f1f1f', plot_bgcolor='#181818')
fig.add_trace(px.scatter(x=np.sin(
    df['setpoint']), y=-np.cos(df['setpoint'])).data[0]).update_traces(marker=dict(size=30))
fig.add_trace(px.scatter(df, x='x', y='y', color='t').data[0])
fig.show()

# Double Pendulum Control

In [4]:
errors_dict = {0: np.zeros(2)}
control_signal = {0: 0}


def u(y, t, setpoint, Kp, Kd, Ki):
    """Pendulum control law
    Args:
        y (n x 1 np.ndarray): [theta1, omega1, theta2, omega2]
        t (float): time
        setpoint (m x 1 np.ndarray): [setpoint_theta1, setpoint_theta2] setpoint for theta1 and theta2
        Kp (2 x 1 np.ndarray): proportional gain for omega_dot1 and omega_dot2
        Kd (2 x 1 np.ndarray): derivative gain for omega_dot1 and omega_dot2
        Ki (2 x 1 np.ndarray): integral gain for omega_dot1 and omega_dot2
    Returns:
        n x 1 np.ndarray: control input
    """
    theta1 = y[0]
    theta2 = y[2]

    e = np.array(setpoint) - np.array([theta1, theta2])
    errors_array = np.asarray(list(errors_dict.values()))
    e_dot = (e - errors_array[-1]) / dt
    e_sum = np.sum(errors_array, axis=0)

    errors_dict[t] = e
    control_signal[t] = Kp * e + Kd * e_dot + Ki * e_sum * dt

    return control_signal[t]


def f(y, t, c, u):
    """Double pendulum ODE
    Args:
        y (n x 1 np.ndarray): [theta1, omega1, theta2, omega2]
        t (float): time
        c (float): damping coefficient
        u (float): control input
    Returns:
        n x 1 np.ndarray: [dtheta1/dt, domega1/dt, dtheta2/dt, domega2/dt]
    """
    theta1, omega1, theta2, omega2 = y

    # equations of motion
    omega1_dot = ((omega1**2*np.sin(theta2-theta1)*np.cos(theta2-theta1)
                   + np.sin(theta2)*np.cos(theta2-theta1)
                   + omega2**2*np.sin(theta2-theta1)
                   - np.sin(theta1))
                  / (2 - np.cos(theta2-theta1)**2) - c*omega1) + u[0]

    omega2_dot = ((-omega2**2*np.sin(theta2-theta1)*np.cos(theta2-theta1)
                   + (np.sin(theta1)*np.cos(theta2-theta1)
                   - omega1**2*np.sin(theta2-theta1)
                   - np.sin(theta2)))
                  / (2 - np.cos(theta2-theta1)**2) - c*omega2) + u[1]

    return np.array([omega1, omega1_dot, omega2, omega2_dot])



def G(y, t, epsilon):
    """Pendulum diffusion matrix
    Args:
        y (n x 1 np.ndarray): [theta, omega]
        t (float): time
        epsilon (float, optional): maximum diffusion strength. Defaults to 0.1.
    Returns:
        n x n np.ndarray: diffusion matrix
    """
    n = y.shape[0]
    return np.random.uniform(-epsilon, epsilon, size=(n, n))


# pendulum initial conditions
y0 = np.array([np.pi/2, 0, np.pi/4, 0])

# pendulum time span
dt = 0.1
tspan = np.arange(0, 100, dt).round(2)

# parameters
SETPOINT_THETA1 = np.pi
SETPOINT_THETA2 = 0

KP_1 = 15
KP_2 = 15
KP = np.array([KP_1, KP_2])

KD_1 = 15
KD_2 = 15
KD = np.array([KD_1, KD_2])

KI_1 = 1
KI_2 = 1
KI = np.array([KI_1, KI_2])


MAX_DIFFUSION = 1e-2  # maximum diffusion strength
C = 1e-3  # damping

ito = itoint(
    f=lambda y, t: f(y, t, C, u(y, t, [SETPOINT_THETA1, SETPOINT_THETA2], KP, KD, KI)),
    G=lambda y, t: G(y, t, MAX_DIFFUSION),
    y0=y0,
    tspan=tspan)

df = pd.DataFrame(ito, columns=['theta1', 'omega1', 'theta2', 'omega2'])
df['t'] = tspan
df['setpoint_theta1'] = SETPOINT_THETA1
df['setpoint_theta2'] = SETPOINT_THETA2
df['x1'] = np.sin(df['theta1'])
df['y1'] = -np.cos(df['theta1'])
df['x2'] = df['x1'] + np.sin(df['theta2'])
df['y2'] = df['y1'] - np.cos(df['theta2'])
df['error_theta1'] = df['theta1'] - df['setpoint_theta1']
df['error_theta2'] = df['theta2'] - df['setpoint_theta2']

fig = px.scatter(df, x='t', y=['theta1', 'theta2', 'setpoint_theta1', 'setpoint_theta2'])
fig.update_layout(template='plotly_dark', paper_bgcolor='#1f1f1f').show()

fig = px.scatter(df, x='x2', y='y2', color='t', size_max=1, opacity=0.7)
fig.update_layout(template='plotly_dark', paper_bgcolor='#1f1f1f').show()

# Lorenz System Control

In [3]:
errors_dict = {0: np.zeros(3)}
control_signal = {0: np.zeros(3)}

def u(y, t, setpoint, Kp, Kd, Ki):
    """Pendulum control law
    Args:
        y (n x 1 np.ndarray): [x, y, z]
        t (float): time
        setpoint (n x 1 np.ndarray): setpoint for x, y, z
        Kp (n x 1 np.ndarray): proportional gain for x, y, z
        Kd (n x 1 np.ndarray): derivative gain for x, y, z
        Ki (n x 1 np.ndarray): integral gain for x, y, z
    Returns:
        n x 1 np.ndarray: control input
    """
    e = setpoint - y
    errors_array = np.asarray(list(errors_dict.values()))
    e_dot = (e - errors_array[-1]) / dt
    errors_dict[t] = e
    control_signal[t] = Kp * e + Kd * e_dot + Ki * np.sum(errors_array, axis=0)*dt
    return control_signal[t]


def f(y, t, sigma, rho, beta, u):
    """Lorenz system of differential equations
    
    Args:
        y (np.ndarray): [x, y, z] state vector
        t (float): time
        sigma (float): Prandtl number
        rho (float): Rayleigh number
        beta (float): geometric factor
        u (np.ndarray): [u1, u2, u3] control input
        
    Returns:
        np.ndarray: [dx/dt, dy/dt, dz/dt] + u
    """
    x_dot = sigma * (y[1]-y[0])
    y_dot = y[0]*(rho-y[2])-y[1]
    z_dot = y[0]*y[1]-beta*y[2]

    return np.array([x_dot, y_dot, z_dot]) + u

# pendulum diffusion matrix


def G(y, t, epsilon):
    """Pendulum diffusion matrix
    Args:
        y (np.ndarray): [x, y, z] state vector
        t (float): time
        epsilon (float, optional): maximum diffusion strength. Defaults to 0.1.
    Returns:
        n x n np.ndarray: diffusion matrix
    """
    n = y.shape[0]
    return np.random.uniform(-epsilon, epsilon, size=(n, n))


# system initial conditions
y0 = np.array([1, 1, 1])

# system time span
dt = 0.01
tspan = np.arange(0, 50, dt).round(2)

# parameters
SETPOINT_X = 20
SETPOINT_Y = 20
SETPOINT_Z = 20

SIGMA = 10
RHO = 28
BETA = 8/3

# proportional gain
KP_X = 15
KP_Y = 20
KP_Z = 15

# derivative gain
KD_X = 0.7
KD_Y = 0.9
KD_Z = 0.7

# integral gain
KI_X = 10
KI_Y = 5
KI_Z = 10

MAX_DIFFUSION = 1  # maximum diffusion strength

ito = itoint(
    f=lambda y, t: f(y, t, SIGMA, RHO, BETA, u(y, t, np.array([SETPOINT_X, SETPOINT_Y, SETPOINT_Z]), np.array([KP_X, KP_Y, KP_Z]), np.array([KD_X, KD_Y, KD_Z]), np.array([KI_X, KI_Y, KI_Z]))),
    G=lambda y, t: G(y, t, MAX_DIFFUSION),
    y0=y0,
    tspan=tspan)

df = pd.DataFrame(ito, columns=['x', 'y', 'z'])
df['t'] = tspan
df['x_setpoint'] = SETPOINT_X
df['y_setpoint'] = SETPOINT_Y
df['z_setpoint'] = SETPOINT_Z

fig = px.scatter_3d(df, x='x', y='y', z='z', color='t', size_max=1, opacity=0.7)
fig.update_traces(marker=dict(size=1))
fig.update_layout(template='plotly_dark', paper_bgcolor='#1f1f1f',
                  margin=dict(l=0, r=0, b=0, t=0, pad=0))
fig.show()

fig = px.line(df, x='t', y=['x', 'y', 'z', 'x_setpoint', 'y_setpoint', 'z_setpoint'])
fig.update_layout(template='plotly_dark', paper_bgcolor='#1f1f1f')
fig.show()